# Volume 2, Chapter 14: RAG Fundamentals

**Making Network Documentation Searchable with AI**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eduardd76/AI_for_networking_and_security_engineers/blob/master/Volume-2-Practical-Applications/Colab-Notebooks/Vol2_Ch14_RAG_Fundamentals.ipynb)

## What You'll Learn

This notebook demonstrates:
- ✅ Understanding Retrieval-Augmented Generation (RAG)
- ✅ Working with vector embeddings
- ✅ Building a vector database
- ✅ Implementing semantic search
- ✅ Answering questions from documentation
- ✅ Production-grade RAG system
- ✅ Integrating with Chapter 13 documentation

By the end, you'll have a searchable documentation system.

## Setup & Dependencies

In [ ]:
# Install required packages
!pip install -q anthropic chromadb

import os
import json
from datetime import datetime
from pathlib import Path
from typing import Dict, List
from getpass import getpass

# Configure API key
if 'ANTHROPIC_API_KEY' not in os.environ:
    api_key = getpass('Enter your Anthropic API key: ')
    os.environ['ANTHROPIC_API_KEY'] = api_key
else:
    api_key = os.environ['ANTHROPIC_API_KEY']

print("✅ Dependencies installed and API key configured!")

## The Problem We're Solving

You just generated documentation for 500 network devices (Chapter 13). Great!

But now:
- **Engineer needs to know**: "Which routers run BGP in production?"
- **Current process**: Search folder → Open 500 files → Find answer (30 minutes)
- **Better way**: Ask a question → RAG system → Get answer (10 seconds)

**RAG = Retrieval-Augmented Generation**

- **Retrieval**: Find relevant documentation using semantic search
- **Generation**: Use Claude to answer based on retrieved docs
- **Result**: Natural language answers grounded in your actual documentation

## Part 1: Understanding Vector Embeddings

### The Core Concept

An **embedding** is a numerical representation of text:

```
"ip ospf cost 100"
    ↓
[0.234, -0.891, 0.123, 0.456, ..., 0.789]  (1,536 numbers)
    ↓
This vector represents the meaning of the text
```

**Magic property**: Similar texts have similar vectors.

```
"ip ospf cost 100"     → [0.234, -0.891, 0.123, ...]
"ospf interface cost"  → [0.235, -0.890, 0.124, ...]  ← Very similar!

"BGP neighbor 1.2.3.4" → [-0.456, 0.789, 0.234, ...]  ← Different!
```

This lets us do **semantic search** - find similar meanings, not just keywords.

## Part 2: Building a RAG System

Let's build a complete RAG system step by step.

In [ ]:
from anthropic import Anthropic
import chromadb

class DocumentationRAG:
    """Retrieve and answer questions about network documentation."""

    def __init__(self):
        self.client = Anthropic(api_key=os.environ['ANTHROPIC_API_KEY'])
        # Chroma stores embeddings automatically
        self.chroma = chromadb.EphemeralClient()  # In-memory for demo
        self.collection = self.chroma.create_collection(
            name="network_documentation"
        )
        self.doc_count = 0

    def add_documentation(self, documents: List[Dict]):
        """Add documentation to the RAG system.
        
        Args:
            documents: List of dicts with 'id', 'content', 'source'
        """
        
        ids = [d['id'] for d in documents]
        contents = [d['content'] for d in documents]
        metadata = [
            {'source': d['source'], 'device': d.get('device', 'unknown')}
            for d in documents
        ]

        self.collection.add(
            ids=ids,
            documents=contents,
            metadatas=metadata
        )
        
        self.doc_count += len(documents)
        print(f"✓ Added {len(documents)} documents ({self.doc_count} total)")

    def retrieve_relevant_docs(
        self,
        question: str,
        n_results: int = 3
    ) -> List[Dict]:
        """Find relevant documentation for a question."""
        
        results = self.collection.query(
            query_texts=[question],
            n_results=n_results
        )

        # Format results
        retrieved = []
        for doc, metadata in zip(
            results['documents'][0],
            results['metadatas'][0]
        ):
            retrieved.append({
                'content': doc,
                'source': metadata['source'],
                'device': metadata['device']
            })

        return retrieved

    def answer_question(self, question: str, n_results: int = 3) -> Dict:
        """Answer a question using retrieved documentation."""
        
        # Step 1: Retrieve relevant documentation
        docs = self.retrieve_relevant_docs(question, n_results)

        if not docs:
            return {
                "answer": "No relevant documentation found.",
                "sources": []
            }

        # Step 2: Build context from retrieved docs
        context = "\n\n".join([
            f"From {doc['source']} ({doc['device']}):\n{doc['content']}"
            for doc in docs
        ])

        # Step 3: Ask Claude to answer using the context
        prompt = f"""Based on the following network documentation, answer this question:

Question: {question}

Documentation:
{context}

Answer based ONLY on the documentation above. Be specific and concise.
If the answer isn't in the docs, say so."""

        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=500,
            messages=[{
                "role": "user",
                "content": prompt
            }]
        )

        return {
            "answer": response.content[0].text,
            "sources": [doc['source'] for doc in docs]
        }

print("✅ DocumentationRAG class loaded")

## Create Sample Documentation

Let's create some sample network documentation to demonstrate RAG.

In [ ]:
# Sample network documentation
sample_docs = [
    {
        "id": "router-core-01_overview",
        "source": "router-core-01.md",
        "device": "router-core-01",
        "content": """Device: router-core-01
Role: Core Router
Management IP: 192.168.1.1

Routing Protocols: BGP, OSPF
Key Features: HSRP (primary, priority 110), VRF-aware routing, QoS enabled

BGP Configuration:
- Local AS: 65001
- Neighbors: 203.0.113.2 (ISP AS 65002), 10.0.1.2 (Datacenter AS 65100)
- Advertised networks: 10.0.0.0/16, 172.16.0.0/16
- Route redistribution from OSPF with metric 100

OSPF Configuration:
- Process ID: 1
- Router ID: 192.168.1.1
- Areas: 0 (backbone)
- Interface costs: Gi0/0=100, Gi0/1=50, Gi0/2=200"""
    },
    {
        "id": "router-core-02_overview",
        "source": "router-core-02.md",
        "device": "router-core-02",
        "content": """Device: router-core-02
Role: Core Router (Backup)
Management IP: 192.168.1.2

Routing Protocols: BGP, OSPF
Key Features: HSRP (standby, priority 100), VRF-aware routing, QoS enabled

BGP Configuration:
- Local AS: 65001
- Neighbors: 203.0.113.3 (ISP AS 65002), 10.0.1.3 (Datacenter AS 65100)
- Advertised networks: 10.0.0.0/16, 172.16.0.0/16
- Route redistribution from OSPF with metric 100

OSPF Configuration:
- Process ID: 1
- Router ID: 192.168.1.2
- Areas: 0 (backbone)
- Interface costs: Gi0/0=100, Gi0/1=50, Gi0/2=200"""
    },
    {
        "id": "switch-dist-01_vlan",
        "source": "switch-dist-01.md",
        "device": "switch-dist-01",
        "content": """Device: switch-dist-01
Role: Distribution Switch

VLAN Configuration:
- VLAN 10: Management (10.10.1.0/24)
- VLAN 20: Users (10.20.1.0/24)
- VLAN 30: Servers (10.30.1.0/24)
- VLAN 100: Uplink (point-to-point)

Spanning Tree:
- Protocol: RSTP
- Root bridge priority: 24576 (bridge priority 4096)
- Port priorities configured for predictable blocking

Port Security:
- Enabled on all access ports
- Maximum 3 MAC addresses per port
- Violation action: shutdown"""
    },
    {
        "id": "firewall-01_acl",
        "source": "firewall-01.md",
        "device": "firewall-01",
        "content": """Device: firewall-01
Role: Perimeter Firewall

Security Policies:
- Management Access: Restricted to 10.0.0.0/16 via SSH only
- Internet Outbound: Allowed with NAT
- Internet Inbound: Only on published services (DNS, HTTP, HTTPS)
- VPN Connections: IPsec tunnels to branch offices

Access Control Lists (ACLs):
- INSIDE_OUT: Permit internal traffic to internet
- OUTSIDE_IN: Deny all except published services
- ADMIN_ACCESS: Restrict admin to subnet 10.10.0.0/16

NAT Configuration:
- Inside global: 203.0.113.0/25
- Inside local: 10.0.0.0/8
- PAT (Port Address Translation) enabled for scalability"""
    },
    {
        "id": "router-branch-01_config",
        "source": "router-branch-01.md",
        "device": "router-branch-01",
        "content": """Device: router-branch-01
Role: Branch Office Router
Location: Branch Office 1

Connectivity:
- WAN: 2 x MPLS circuits to HQ (primary + backup)
- Local LAN: 10.50.1.0/24
- VPN: IPsec tunnel to other branches

Routing Configuration:
- OSPF Area 1 (branch area)
- Default route via HQ core routers
- Static routes for non-OSPF destinations

BGP: Not configured (relies on OSPF)

QoS:
- Voice traffic: Priority queue
- Video: High bandwidth queue
- Data: Standard queue with WRED"""
    }
]

print(f"Created {len(sample_docs)} sample documents")
for doc in sample_docs:
    print(f"  • {doc['device']}: {doc['source']}")

In [ ]:
# Initialize RAG system and add documentation
print("🚀 Initializing RAG system...")
rag = DocumentationRAG()

print("\n📚 Adding documentation...")
rag.add_documentation(sample_docs)

print("\n✅ RAG system ready!")

## Test RAG with Questions

Now let's ask questions about the documentation and see how RAG answers them.

In [ ]:
# Test questions
test_questions = [
    "Which routers run BGP?",
    "What's the OSPF configuration?",
    "What security policies are in place on the firewall?",
    "How is HSRP configured for redundancy?",
    "Which VLANs are configured on the distribution switch?"
]

print("🔍 Testing RAG with sample questions...\n")

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {question}")
    print(f"{'='*80}")
    
    result = rag.answer_question(question, n_results=2)
    
    print(f"\nAnswer:\n{result['answer']}")
    print(f"\nSources: {', '.join(result['sources'])}")

## Advanced RAG Pattern: Confidence Scoring

Know when the system is confident vs uncertain about its answers.

In [ ]:
class ConfidentRAG(DocumentationRAG):
    """RAG with confidence scoring."""

    def answer_with_confidence(self, question: str, n_results: int = 3) -> Dict:
        """Answer question and provide confidence score."""
        
        # Retrieve docs
        results = self.collection.query(
            query_texts=[question],
            n_results=n_results
        )
        
        # Calculate confidence from relevance scores
        if results['distances'][0]:
            # Lower distance = higher confidence
            distances = results['distances'][0]
            avg_distance = sum(distances) / len(distances)
            
            if avg_distance < 0.3:
                confidence = "HIGH"
            elif avg_distance < 0.6:
                confidence = "MEDIUM"
            else:
                confidence = "LOW"
        else:
            confidence = "NONE"
        
        # Get answer
        answer_result = self.answer_question(question, n_results)
        
        return {
            **answer_result,
            "confidence": confidence
        }

# Test with confidence
print("Testing RAG with confidence scoring...\n")
conf_rag = ConfidentRAG()
conf_rag.add_documentation(sample_docs)

test_q = "What security policies are configured?"
result = conf_rag.answer_with_confidence(test_q)

print(f"Question: {test_q}")
print(f"\nAnswer:\n{result['answer']}")
print(f"\nConfidence: {result['confidence']}")
print(f"Sources: {', '.join(result['sources'])}")

## Integration: Chapter 13 → Chapter 14

### How Documentation Flows Through the System

```
Device Configs
    ↓
[Chapter 13] Auto-generate documentation
    ↓
Documentation files (router-core-01.md, etc.)
    ↓
[Chapter 14] Index into vector database
    ↓
RAG System Ready for Questions
    ↓
User asks: "What's configured on router-core-01?"
    ↓
RAG retrieves relevant docs and answers
```

### Complete Workflow

```python
# 1. Generate docs (Chapter 13)
generator = ConfigDocumentationGenerator(api_key)
generator.generate_complete_documentation(config, hostname)

# 2. Index for RAG (Chapter 14)
rag = DocumentationRAG()
for doc_file in Path("./docs").glob("*.md"):
    with open(doc_file) as f:
        rag.add_documentation([{
            "id": doc_file.stem,
            "content": f.read(),
            "source": doc_file.name
        }])

# 3. Answer questions
result = rag.answer_question("Which routers run BGP?")
print(result["answer"])
```

## Best Practices for Production RAG

### 1. Document Chunking
Break large documents into smaller chunks for better retrieval:
- **Too small** (100 words): Loses context
- **Too large** (5000 words): Retrieves irrelevant material
- **Optimal** (1000-1500 words): Balanced context

### 2. Query Expansion
Ask multiple related questions to improve search:
```
User: "How do we route to AWS?"
System asks:
  - "BGP configuration for AWS"
  - "AWS peering configuration"
  - "AWS routing policies"
Finds more relevant results
```

### 3. Result Ranking
Use Claude to re-rank results for relevance:
```
Initial retrieval: 10 documents
Claude ranks by relevance to question
Return top 3 most relevant
```

### 4. Confidence Scoring
Always include confidence in answers:
- **HIGH**: Exact matches found
- **MEDIUM**: Relevant but not perfect
- **LOW**: Limited relevant documentation

### 5. Feedback Collection
Track which answers were helpful:
- Was this answer helpful? ✓/✗
- Use feedback to improve system
- Identify documentation gaps

## Try It Yourself

Ask your own questions about the documentation!

In [ ]:
# Ask your own question
your_question = "Which devices have redundancy configured?"  # Change this!

print(f"\n📝 Question: {your_question}\n")
print("🔍 Retrieving documentation...\n")

result = rag.answer_question(your_question, n_results=3)

print(f"Answer:\n{result['answer']}")
print(f"\nSources:\n")
for source in result['sources']:
    print(f"  • {source}")

## Summary

### What You've Learned

✅ **RAG Basics** - Retrieval + Generation for grounded answers  
✅ **Vector Embeddings** - Semantic representation of text  
✅ **Vector Databases** - Chroma for semantic search  
✅ **RAG Implementation** - Complete working system  
✅ **Confidence Scoring** - Know when to trust the system  
✅ **Integration** - Connect with Chapter 13 documentation  
✅ **Best Practices** - Production-grade patterns  

### Key Metrics

| Metric | Before RAG | After RAG |
|--------|-----------|----------|
| Time to find answer | 10+ min | <1 min |
| Answer accuracy | Variable | Grounded in docs |
| Documentation discovery | Manual | Automatic |

### Next Steps

1. **Load your documentation** - Use docs from Chapter 13
2. **Create RAG system** - Index all documents
3. **Test with real questions** - Verify accuracy
4. **Deploy as API** - Make available to team
5. **Collect feedback** - Improve over time

### Resources

📚 [Chroma Vector Database](https://docs.trychroma.com/)  
📚 [Anthropic API Docs](https://docs.anthropic.com/)  
📚 [Chapter 13: Network Documentation](https://github.com/eduardd76/AI_for_networking_and_security_engineers)  

---

**Happy searching! 🔍**